In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

#holas, ORRAR este comentario
class Scraper():
    base_url = 'https://listado.mercadolibre.com.mx/'
    def scraping(self):
        # User search
        product_name = input("\nProducto: ")
        # Clean the user input
        cleaned_name = product_name.replace(" ", "-").lower()
        # Create the urls to scrap
        urls = [self.base_url + cleaned_name]

        page_number = 50
        for i in range(0, 10000, 50):
            urls.append(f"{self.base_url}{cleaned_name}_Desde_{page_number + 1}_NoIndex_True")
            page_number += 50
        # create a list to save the data
        self.data = []
        # create counter
        c = 1
        # Iterate over each url
        for i, url in enumerate(urls, start=1):
            # Get the html of the page
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            # take all posts
            content = soup.find_all('li', class_='ui-search-layout__item')
            # Check if there's no content to scrape
            if not content:
#                print("\nTermino el scraping.")
                break
#            print(f"\nScrapeando pagina numero {i}. {url}")
            # iteration to scrape posts
            for post in content:
                post_link = post.find("a")["href"]
                post_data = {
                    # "title": title,
               #     "price": price,
                    "post link": post_link,
              #      "image link": img_link            
                }
                # save the dictionaries in a list
                self.data.append(post_data)
                c += 1

    def export_to_csv(self):
        # export to a csv file
        df = pd.DataFrame(self.data)
        df.to_csv(r"ML_data.csv", sep=";")

if __name__ == "__main__":
    s = Scraper()
    s.scraping()
    s.export_to_csv()





In [5]:

## LIMPEZA LINKS
import re
# Cargar el archivo CSV que contiene los links
df = pd.read_csv("ML_data.csv")  # Asegúrate de usar el nombre correcto de tu archivo

# Definir una función para extraer la parte deseada del enlace
def extract_code(link):
    # Intentar extraer el código entre '/p/' y '#wid%'
    match = re.search(r'/p/([^#]*)#polycard', link)
    if match:
        return match.group(1)
    
    # Si no se encuentra '/p/', buscar el código entre 'mx/MLM-' y el siguiente '-'
    match_mlm = re.search(r'mx/MLM-(\d+)-', link)
    if match_mlm:
        return f"MLM{match_mlm.group(1)}"
    
    # Retornar None si no se encuentra ninguno de los patrones
    return None

# Definir una función para crear el nuevo link con el formato deseado
def create_new_link(code):
    if code:
        return f"https://www.mercadolibre.com.mx/noindex/catalog/reviews/{code}?noIndex=true&access=view_all&modal=true&controlled=true"
    return None

# Aplicar las funciones para extraer y crear el nuevo link
df['CodigoExtraido'] = df[';post link'].apply(extract_code)
df['NuevoLink'] = df['CodigoExtraido'].apply(create_new_link)

# Conservar solo la columna 'NuevoLink' y eliminar filas vacías
df_final = df[['NuevoLink']].dropna()

# Guardar el resultado en el archivo original
df_final.to_csv("ML_data2.csv", index=False)